In [14]:
import numpy as np
import healpy as hp
from numba import jit, njit, prange, set_num_threads
from numba.typed import List, Dict
from tqdm.notebook import tqdm
from helper_funcs import *
import bispec_calculator as bispec

In [15]:
def sort_alms(alms, lmax):
    '''
    Sorts healpix alm's by \ell instead of m given a fortran90 array output from hp.map2alm.

    Parameters
    ----------
    alms : fortran90 healpix alm array from hp.map2alm
    num_ls : number of l's in alms array
    
    Returns
    ----------
    sorted_alms : alm dictionary keyed by ell values with numpy arrays 
                    consisting of the corresponding m values.
    '''
    start = 0
    sorted_alms = {}

    for l in range(lmax + 1):
        sorted_alms[l] = np.zeros(2*l+1, dtype=np.cdouble)

    for m in range(lmax + 1):
        num_ms = lmax + 1 - m
        ms = alms[start:num_ms + start]
        start = num_ms + start
        m_sign = (-1)**m
        for l in range(num_ms):
            idx = m + l
            sorted_alms[idx][m] = ms[l]
            if m != 0:
                sorted_alms[idx][-m] = m_sign * np.conj(ms[l])
    
    return sorted_alms

In [16]:
def unsort_alms(sorted_alms):
    """
    Unsorts a sorted_alm dictionary from sorted_alms into a Fortran90
    array that can be processed by HEALPix.

    Inputs:
        sorted_alms (dict[ndarray]) : sorted alms from sort_alms

    Returns:
        (ndarray) : unsorted alms for HEALPix
    """
    ells = np.array(list(sorted_alms.keys()))
    lmax = ells[-1]
    unsorted_alms = np.zeros(np.sum(ells + 1), dtype=np.complex128)

    start = 0
    for m in range(lmax + 1):
        for ell in range(m, lmax + 1):
            unsorted_alms[start + ell] = sorted_alms[ell][m]
        start += lmax - m
    
    return unsorted_alms

In [17]:
def filter_map_binned(sorted_alms, i_bins, nside=1024):
    """
    Map alms that have been binned and filtered.

    Inputs:
        sorted_alms (dict) : dictionary of sorted alms
        i_bins (ndarray) : ibins
    
    Returns:
        (ndarray) : healpix map of filtered alms
    """
    lmax = max(sorted_alms.keys())
    filtered_alms = {}
    for ell in range(lmax + 1):
        filtered_alms[ell] = np.zeros(2 * ell + 1, dtype=np.cdouble)

    for ell in i_bins:
        filtered_alms[ell] = sorted_alms[ell]
    
    return hp.alm2map(unsort_alms(filtered_alms), nside=nside)

In [18]:
lmax = 1024
ells = np.arange(1, lmax+1)

cls = (ells + 0.0)**(-3.)

theory_map, alms = hp.sphtfunc.synfast(cls=cls, nside=1024, lmax=lmax, alm=True)

In [19]:
lmin_bin = 2
lmax_bin = 10

bin1, _, _ = bispec.select_bins(lmin_bin, lmax_bin)

In [20]:
lmax = hp.Alm.getlmax(alms.size)

In [21]:
bin1_max = bin1.max()

In [22]:
bin1

array([2., 3., 4., 5., 6., 7., 8., 9.])

In [23]:
ell = 9

indices = hp.Alm.getidx(lmax, ell, np.arange(-ell, ell + 1)).astype(int)

In [24]:
ms = np.arange(-ell, ell + 1)
indices = hp.Alm.getidx(lmax, ell, ms)

In [25]:
indices

array([-9252, -8219, -7187, -6156, -5126, -4097, -3069, -2042, -1016,
           9,  1033,  2056,  3078,  4099,  5119,  6138,  7156,  8173,
        9189])

In [32]:
def filter_alms_no_dict(alms, bins):
    """
    Filters alms without using dictionaries.

    Parameters:
        alms (ndarray) : alms
        bins (ndarray) : bins
    
    Returns:
        filtered_alms (ndarray) : filtered alms
    """
    lmax = hp.Alm.getlmax(alms.size)
    filtered_alms = np.zeros_like(alms)
    for ell in bins:
        indices = hp.Alm.getidx(lmax, ell, ms).astype(int)
        filtered_alms[indices] = alms[indices]
    return filtered_alms

In [33]:
filtered_alms = filter_alms_no_dict(alms, bin1)